# Hit Calling for kChip Lysis-Antagonism Screen
**Written:** 20220811\
**Last Updated:** 20220831

# Inputs & Imports

## configurable 

In [ ]:
file_id = 'full_batch_'

# path to scripts
script_reroute = '../path/'

# total imaging tp
num_tp = 48

In [ ]:
# half hour tps
exact_tp = [i/2 for i in range(num_tp)]

In [ ]:
import os
out_path = './output/image_analysis/'+file_id
core_path =  './output/core/'+file_id
cc_path =  './output/coculture/'+file_id

os.makedirs('./output/', exist_ok=True)
os.makedirs('./output/core/', exist_ok=True)
os.makedirs('./output/coculture/', exist_ok=True)

## standard

## packages & scripts

In [ ]:
import re
import glob
import numpy as np
import pandas as pd
import scipy
import scipy.stats as stats
from sklearn.metrics import auc
from statsmodels.stats.multitest import multipletests
import itertools

import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline

import sys
sys.path.insert(1, script_reroute)
import sytox_scripts.bootstrap_and_z as bsz
import sytox_scripts.supplementary as helper
import sytox_scripts.cocultures as cocultures

In [ ]:
# plotting style
plot_path = script_reroute+'sytox_scripts/plotting_parameters.py'
%run $plot_path

# Significance
- null population = mono/mono "self-cocultures" as opposed to
- experimental population = polymicrobial cocultures

## standardize scores
- lysis score / lysis score's standard error

In [ ]:
non_kinetic = pd.read_csv(cc_path+'summarized_cocultures_nonkinetic.csv', index_col=0)

In [ ]:
sig_nk = helper.standardize_dAUC_scores(non_kinetic)

In [ ]:
sig_nk.to_csv(cc_path+'summarized_cocultures_nonkinetic_standardizeScores.csv')
sig_nk

## calculate p-values 
- perform a right-tailed test using standardized lysis scores and a bootstrapped (10k) null distribution 
- FDR-correct with BH procedure

In [ ]:
sig_nk = pd.read_csv(cc_path+'summarized_cocultures_nonkinetic_standardizeScores.csv', index_col=0)
null_nk_st = helper.get_null_pop(sig_nk)
exp_nk_st = helper.get_exp_pop(sig_nk)

In [ ]:
null_bs = bsz.boot_array(null_nk_st.dAUC_by_SE, bs_size=10000)

In [ ]:
null_bs_all = np.concatenate(null_bs.values) # for distribution 

In [ ]:
null_bs_all.shape

In [ ]:
def calc_pval_bs(null_array, score, C=1):
    ''' Right-tailed test, constant to prevent zero. '''
    null = pd.DataFrame(null_array)
    return (null[null[0] >= score].shape[0] + C)/null.shape[0]

def get_all_pvals_bs(df, null_array, score_col='dAUC_by_SE'):
    '''
    Returns a df with p-values from t-distribution of null,
    FDR-corrected p-values and neg/neg_log10 transformed p-values.
    '''
    df['pval'] = [calc_pval_bs(null_array, t) for t in df[score_col]]
    df['pval_fdr'] = multipletests(df.pval, method='fdr_bh')[1]
    df['neg_pval_fdr'] = [-p for p in df. pval_fdr]
    df['neglog_pval'] = [-np.log(p) for p in df.pval_fdr]
    
    return df

In [ ]:
# or run with python script as it's a slow process
bs_sig = get_all_pvals_bs(sig_nk, null_bs_all)

In [ ]:
bs_sig.to_csv(cc_path+'summarized_cocultures_nonkinetic_w_bs_pvals.csv')

# Threshold Hit Calling

In [ ]:
full_co_df = pd.read_csv(cc_path+'summarized_cocultures_kinetic.csv', index_col=0)
non_kinetic = pd.read_csv(cc_path+'summarized_cocultures_nonkinetic.csv', index_col=0)
sig_nk = pd.read_csv(cc_path+'summarized_cocultures_nonkinetic_w_bs_pvals.csv',index_col=0)

## volcano plot of all conditions

In [ ]:
def plot_volcano(df, xcol, ycol, 
                 cutoff_on=False, x_cut='', y_cut='',
                 x_label='dAUC score', y_label='-log10(P-value)', 
                 fig_w=7, fig_h=7, save_dir=cc_path, save_desc=''):

    plt.figure(figsize=(fig_w,fig_h))
    plt.scatter(df[xcol], df[ycol], alpha=0.1, c='gray')
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    
    if cutoff_on == True:
        plt.axhline(y=y_cut, alpha=0.1, c='red')
        plt.axvline(x=x_cut, alpha=0.1, c='red')
    
    plt.savefig(save_dir+'volcano_'+save_desc+'.png')
    
    return

In [ ]:
plot_volcano(sig_nk, 'dAUC_score', 'neglog_pval', save_desc='neglog_pval')

## apply cutoffs & save .csv's of hits

In [ ]:
def apply_hit_cutoff(df, dAUC_cut, pval_cut, 
                     dAUC_col = 'dAUC_score', 
                     pval_col= 'pval_fdr'):
    
    return df[(df[dAUC_col] >= dAUC_cut) & (df[pval_col] <= pval_cut)]

In [ ]:
# call with error-adjusted empirical lysis scores 
hits_nk = apply_hit_cutoff(sig_nk, dAUC_cut=0.1, dAUC_col='dAUC_score_emp_adj', pval_cut=0.05)
hits_nk.shape

In [ ]:
print(hits_nk.dAUC_score_emp_adj.max(), hits_nk.dAUC_score_emp_adj.min())

In [ ]:
hits_nk.to_csv(cc_path+'summarized_hits_nonkinetic.csv')

In [ ]:
labels_df = hits_nk[['chip_ID', 'Label_left', 'Label_right', 'Left_simple', 'Right_simple', 
                     'Labels_combo', 'Combo_simple', 'full_ID']].reset_index(drop=True)
labels_df.to_csv(cc_path+'hit_strain_labels_all.csv')

In [ ]:
def pull_final_hits_kinetic(labels_df, source_df, save_dir=cc_path, save_desc=''):
    '''
    Return a .csv with the full summary & calculations for all final called hits.
    
    Inputs:
        labels_df/source_df: dataframes after labels have been concatenated 
    '''
    hits_df = source_df[source_df.full_ID.isin(labels_df.full_ID)].reset_index(drop=True)
    hits_df.to_csv(save_dir+'summarized_hits'+save_desc+'.csv')
    
    return hits_df 

In [ ]:
hits_df = pull_final_hits_kinetic(labels_df, full_co_df, save_desc='_kinetic')

## view hit curves

In [ ]:
labels_df = pd.read_csv(cc_path+'hit_strain_labels_all.csv', index_col=0)
hits_df = pd.read_csv(cc_path+'summarized_hits_kinetic.csv', index_col=0)

In [ ]:
def plot_all_hits(labels_df, source_df, hr_fraction=0.5, sum_col='mono_sum_RFU_adj',
                  plots_per_page=45, plots_y=9, plots_x=5, fig_w=20, fig_h=25, 
                  rfu_lim_high=1000, rfu_lim_low=500, fold_col = 'fold_change_adj',
                  save_dir=cc_path, filename='hit_cocultures_coplots', 
                  split_by='_', multi=False, position1=1, position2=3):
    '''
    NOTE: use the unfiltered file to grab all timepoints, otherwise could skip (e.g. early tp)
    Plotted mono_sum is error-adjusted.
    
    Separate onto multiple pages to better view.
    
    On a single 20x25 page, fit 5x9 plots reasonably so divides by 45.
    '''
    # separate all hits across pages
    starting_indices = list(range(0, labels_df.shape[0], plots_per_page))
    
    # to plot in hours
    time_base = source_df.tp.unique()
    times = [t*hr_fraction for t in time_base] 
    
    # save all to master PDF
    with PdfPages(save_dir+filename+'.pdf') as pdf:

        # plot each coculture
        for ind in starting_indices:
            # RFUs per page
            fig = plt.figure(figsize=(fig_w, fig_h))
            fig.subplots_adjust(hspace=0.4, wspace=0.4)
            
            fig2 = plt.figure(figsize=(fig_w, fig_h))
            fig2.subplots_adjust(hspace=0.4, wspace=0.4)

            # respective fold-changes on next page
            fig3 = plt.figure(figsize=(fig_w, fig_h))
            fig3.subplots_adjust(hspace=0.4, wspace=0.4)

            for i, n in enumerate(labels_df.iloc[ind:ind+plots_per_page].index):
                # pull all data for single coculture
                
                # chip_ID enabled for batch
                chip_df = source_df[source_df.chip_ID.str.contains(labels_df.chip_ID[n])]
                
                sub_df = helper.extract_combos(chip_df, labels_df.Label_left[n],
                                               labels_df.Label_right[n])
                
                # for cleaner title
                trimmed_left = helper.trim_label_name(labels_df.Label_left[n], multi=True, 
                                                      position1=position1, position2=position2)
                trimmed_right = helper.trim_label_name(labels_df.Label_right[n], multi=True, 
                                                       position1=position1, position2=position2)
                combo = trimmed_left + '/' + trimmed_right + '(%s)' %labels_df.chip_ID[n]

                # coplot RFUs - high lim
                ax = fig.add_subplot(plots_y, plots_x, i+1)
                ax.errorbar(times, y=sub_df.co_RFU, yerr=sub_df.co_error, alpha=0.5)
                ax.errorbar(times, y=sub_df[sum_col], yerr=sub_df.mono_sum_error, alpha=0.5)
                ax.errorbar(times, y=sub_df.left_RFU, yerr=sub_df.left_error, alpha=0.5)
                ax.errorbar(times, y=sub_df.right_RFU, yerr=sub_df.right_error, alpha=0.5)
                ax.set_ylim(0, rfu_lim_high)
                ax.set_title(combo)
                
                # coplot RFUs - low lim
                ax2 = fig2.add_subplot(plots_y, plots_x, i+1)
                ax2.errorbar(times, y=sub_df.co_RFU, yerr=sub_df.co_error, alpha=0.5)
                ax2.errorbar(times, y=sub_df[sum_col], yerr=sub_df.mono_sum_error, alpha=0.5)
                ax2.errorbar(times, y=sub_df.left_RFU, yerr=sub_df.left_error, alpha=0.5)
                ax2.errorbar(times, y=sub_df.right_RFU, yerr=sub_df.right_error, alpha=0.5)
                ax2.set_ylim(0, rfu_lim_low)
                ax2.set_title(combo)

                # fold-changes on next page 
                ax3 = fig3.add_subplot(plots_y, plots_x, i+1)
                ax3.plot(times, sub_df[fold_col])
                ax3.set_title(combo)

            fig.legend(['coculture', 'mono sum', 'left', 'right'], loc='right')
            fig2.legend(['coculture', 'mono sum', 'left', 'right'], loc='right')
            fig3.legend(['experimental/expected'], loc='right')
            
            fig.text(0.5, 0.08, 'time (h)', ha='center', fontsize=20)
            fig.text(0.08, 0.5,'RFU', va='center', rotation='vertical', fontsize=20)
            fig2.text(0.5, 0.08, 'time (h)', ha='center', fontsize=20)
            fig2.text(0.08, 0.5,'RFU', va='center', rotation='vertical', fontsize=20)
            fig3.text(0.5, 0.08, 'time (h)', ha='center', fontsize=20)
            fig3.text(0.08, 0.5, 'fold-change (experimental/expected)', va='center', rotation='vertical', fontsize=20)

            pdf.savefig(fig)
            pdf.savefig(fig2)
            pdf.savefig(fig3)
            
        plt.close()
    
    return

In [ ]:
# sort to group same type of combination (labels) 
labels_sorted = labels_df.sort_values(by=['Left_simple', 'Right_simple', 'Label_left', 'Label_right', 'chip_ID'])

In [ ]:
plot_all_hits(labels_sorted, hits_df, rfu_lim_high=3000, filename='hit_cocultures_coplots_sorted')

# Chip Overlap Hit Calling
- final hit must be found on at least two chips (inocula/ratio-level considered separately; i.e. not simplified labels)

In [ ]:
labels_df = pd.read_csv(cc_path+'hit_strain_labels_all.csv', index_col=0)
hits_df = pd.read_csv(cc_path+'summarized_hits_kinetic.csv', index_col=0)
hits_nk = pd.read_csv(cc_path+'summarized_hits_nonkinetic.csv', index_col=0)

In [ ]:
hits_nk = hits_nk.sort_values(by=['Left_simple', 'Right_simple', 'Label_left', 'Label_right', 'chip_ID'])

## condition must appear on two chips

In [ ]:
overlap = hits_nk[hits_nk.groupby('Labels_combo')['Labels_combo'].transform('size').gt(1)] # greater than 1

In [ ]:
overlap.shape

In [ ]:
overlap.to_csv(cc_path+'summarized_hits_nonkinetic_chipOverlap.csv')

In [ ]:
labels_overlap = overlap[['chip_ID', 'Label_left', 'Label_right', 'Left_simple', 'Right_simple', 
                          'Labels_combo', 'Combo_simple', 'full_ID']].reset_index(drop=True)

# sort to group same type of combination 
overlap_sorted = labels_overlap.sort_values(by=['Left_simple', 'Right_simple', 'Label_left', 'Label_right', 'chip_ID'])
overlap_sorted.to_csv(cc_path+'hit_strain_labels_overlap_sorted.csv')

In [ ]:
overlap_kinetic = pull_final_hits_kinetic(overlap, hits_df, save_desc='_kinetic_chipOverlap')

## plot final hits
- meets lysis score & adjusted-pvalue threshold
- found on at least 2 kChips

In [ ]:
plot_all_hits(labels_overlap, overlap_kinetic, rfu_lim_high=3000, filename='hit_cocultures_coplots_sorted_overlap')

# Best Scores Per Unique Coculture
**2 methods to track best conditions per coculture** 
- highest score among all biological replicates & ratios ("max")
- mean of all scores which passed, error-adjusted ("best")

Produces dataframe with best scores, ranked scores, and associated inocula ratios.

In [ ]:
overlap_nk = pd.read_csv(cc_path+'summarized_hits_nonkinetic_chipOverlap.csv', index_col=0)
overlap_kinetic = pd.read_csv(cc_path+'summarized_hits_kinetic_chipOverlap.csv', index_col=0)

## call most robust score among inocula ratios per hit conditions

In [ ]:
def call_robust_hit(df, score_col='dAUC_score_emp_adj'):
    ''' 
    Method 1: absolute max of all scores
    Method 2: mean score from all passing conditions, error-adjusted by the std of all passing scores
    '''
    all_subdfs = []
    
    for combo in df.Combo_simple.unique():
        subdf = df[df.Combo_simple.str.contains(combo)].reset_index(drop=True)
        
        # method 2 -- error-adjusted mean, "best" score
        sub_mean = subdf.groupby(['Labels_combo']).mean()[score_col]
        sub_std = subdf.groupby(['Labels_combo']).std()[score_col]
        sub_adj = sub_mean - sub_std 
        
        calc_df = pd.DataFrame([sub_adj, sub_mean, sub_std], ['adjusted_score','mean', 'std']).T
        calc_df['ratio'] = [('_').join(i.split('_')[2::3]) for i in calc_df.index]
        calc_df = calc_df.sort_values('adjusted_score', ascending=False)
        
        hh = calc_df[calc_df.ratio.str.contains('high_high')][['adjusted_score','mean','std']].rename(columns={'adjusted_score': 'hh_adj', 'mean': 'hh_mean', 'std':'hh_std'}).reset_index(drop=True)
        hl = calc_df[calc_df.ratio.str.contains('high_low')][['adjusted_score','mean','std']].rename(columns={'adjusted_score': 'hl_adj', 'mean': 'hl_mean', 'std':'hl_std'}).reset_index(drop=True)
        lh = calc_df[calc_df.ratio.str.contains('low_high')][['adjusted_score','mean','std']].rename(columns={'adjusted_score': 'lh_adj', 'mean': 'lh_mean', 'std':'lh_std'}).reset_index(drop=True)
        ll = calc_df[calc_df.ratio.str.contains('low_low')][['adjusted_score','mean','std']].rename(columns={'adjusted_score': 'll_adj', 'mean': 'll_mean', 'std':'ll_std'}).reset_index(drop=True)
        
        # method 1 -- max score
        max_score = subdf[score_col].max()
        max_ID = subdf.full_ID[subdf[score_col].idxmax(axis=0)]
        max_ratio = ('_').join(max_ID.split('_')[3::3])
        
        # some ratios may be empty -- to prevent df join from failing
        full = pd.DataFrame({'Combo_simple': [combo], 
                             'Left_simple': [combo.split('_')[0]], 'Right_simple': [combo.split('_')[1]],
                             'best_ratio': [calc_df.ratio[0]], 'best_score': [calc_df.adjusted_score[0]],
                             'ratio_ranking': [list(calc_df.ratio)], 'score_ranking': [list(calc_df.adjusted_score)],
                             'max_score': [max_score], 'max_ID': max_ID, 'max_ratio': [max_ratio]})
        
        full['ratio_match'] = [True if max_ratio == calc_df.ratio[0] else False]
        
        full = full.join([hh,hl,lh,ll])
        all_subdfs.append(full)

    return pd.concat(all_subdfs)

In [ ]:
robust_scores = call_robust_hit(overlap_nk)
robust_scores

In [ ]:
robust_scores.to_csv(cc_path+'summarized_ranked_scores_per_coculture.csv')

In [ ]:
# can load & maintain lists
robust_scores.to_pickle(cc_path+'summarized_ranked_scores_per_coculture.pkl')